## Import Packages

In [1]:
import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random
import json

from PIL import Image, ImageOps, ImageEnhance
import numbers

import torchattacks
from torchattacks import CW

import matplotlib.pyplot as plt
%matplotlib inline

## Normalize Dataset

In [2]:
#Creating dataset using torch dataloaders
batch_size_train = 128
batch_size_test = 1000
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))]) # Normalizing dataset

# Training dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=transform),
  batch_size=batch_size_train, shuffle=True)

# Test dataset
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=transform),
  batch_size=batch_size_test, shuffle=True)

# Initialize GPU
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


## Initial Model Structure

In [3]:
# Initial model structure
class HNet(nn.Module):    
    def __init__(self):
        super(HNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x     

## SubModel Structure

In [4]:
# Submodel Structure for training residual
class NHNet(nn.Module):    
    def __init__(self):
        super(NHNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x   

## Initialize Data Structures

In [16]:
# Initialize some data structures to store useful data
train_losses = []
train_counter = []
test_losses = []

## Initialize Parameters, optimizer and loss function

In [15]:
# Create the initial model
initial_model = HNet()

#Create the optimizer for the initial model
optimizer = optim.Adam(initial_model.parameters(), lr=0.003)

# Create Loss function for the intial model
criterion = nn.CrossEntropyLoss()

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Change model into cuda mode
if torch.cuda.is_available():
    initial_model = initial_model.cuda()
    criterion = criterion.cuda()

## Training Function for Initial Model

In [7]:
# Initial Model Training Function
def train(epoch):
    initial_model.train()
#     exp_lr_scheduler.step()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = initial_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        # torch.save(initial_model.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/model.pth')
        # torch.save(optimizer.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/optimizer.pth')
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

## Evaluation Function for Intial Model

In [8]:
#Initial Model Evaluating Function
def evaluate(data_loader):
    initial_model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
        
            output = initial_model(data)
        
            loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    test_losses.append(loss)    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

## Start Training the Initial Model

In [17]:
# Start Training
n_epochs = 65

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.602848
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.549241
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.581888
Train Epoch: 0 [51200/60000 (85%)]	Loss: 1.536988


C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':



Average loss: 1.5576, Accuracy: 54379/60000 (90.632%)

Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.562668
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.532746
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.530872
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.574295

Average loss: 1.5313, Accuracy: 55880/60000 (93.133%)

Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.515534
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.524624
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.525929
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.502936

Average loss: 1.5157, Accuracy: 56773/60000 (94.622%)

Train Epoch: 3 [12800/60000 (21%)]	Loss: 1.501767
Train Epoch: 3 [25600/60000 (43%)]	Loss: 1.551804
Train Epoch: 3 [38400/60000 (64%)]	Loss: 1.538258
Train Epoch: 3 [51200/60000 (85%)]	Loss: 1.534766

Average loss: 1.5180, Accuracy: 56624/60000 (94.373%)

Train Epoch: 4 [12800/60000 (21%)]	Loss: 1.514447
Train Epoch: 4 [25600/60000 (43%)]	Loss: 1.528231
Train Epoch: 4 [38400/60000 (64%)]	Loss: 1.516948
Train Epoch: 4 [51200/6000

Train Epoch: 32 [51200/60000 (85%)]	Loss: 1.508222

Average loss: 1.5023, Accuracy: 57525/60000 (95.875%)

Train Epoch: 33 [12800/60000 (21%)]	Loss: 1.508074
Train Epoch: 33 [25600/60000 (43%)]	Loss: 1.547074
Train Epoch: 33 [38400/60000 (64%)]	Loss: 1.548809
Train Epoch: 33 [51200/60000 (85%)]	Loss: 1.521500

Average loss: 1.5059, Accuracy: 57314/60000 (95.523%)

Train Epoch: 34 [12800/60000 (21%)]	Loss: 1.508026
Train Epoch: 34 [25600/60000 (43%)]	Loss: 1.484142
Train Epoch: 34 [38400/60000 (64%)]	Loss: 1.492401
Train Epoch: 34 [51200/60000 (85%)]	Loss: 1.523629

Average loss: 1.5080, Accuracy: 57187/60000 (95.312%)

Train Epoch: 35 [12800/60000 (21%)]	Loss: 1.507460
Train Epoch: 35 [25600/60000 (43%)]	Loss: 1.491980
Train Epoch: 35 [38400/60000 (64%)]	Loss: 1.514934
Train Epoch: 35 [51200/60000 (85%)]	Loss: 1.492401

Average loss: 1.5127, Accuracy: 56892/60000 (94.820%)

Train Epoch: 36 [12800/60000 (21%)]	Loss: 1.508026
Train Epoch: 36 [25600/60000 (43%)]	Loss: 1.484588
Train Epoch

Train Epoch: 64 [25600/60000 (43%)]	Loss: 1.500212
Train Epoch: 64 [38400/60000 (64%)]	Loss: 1.523651
Train Epoch: 64 [51200/60000 (85%)]	Loss: 1.518947

Average loss: 1.5082, Accuracy: 57166/60000 (95.277%)



## Save or Load the Initial Model

In [18]:
# Save this model so that I dont have to train again in the future
torch.save(initial_model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')

In [7]:
# Load the model from local file
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

HNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

## Calculatee mse residual and normalize the result to values between 0 ~ 1

In [10]:
# Function used to calculate mse residual based on the wiki
def mseresidual(y, F):
    residual = y - F
    absolute = torch.abs(residual)
    residual = residual / torch.max(absolute)
    return residual

## Actual Code for gradient Boosting

In [11]:
#Main Function used to train and find optimal gamma for submodels
#input: intial model that's already trained
#M is number of submodels needed to be trained
def GradientBoosting(initial_model, M):
    gamma_exp = torch.ones([M], dtype = torch.float64) # used to hold the final optimized gamma
    models = [] # used to hold all the models
    residual_list = [] # used to hold the residual of each batch calculated
    for m in range(M):
        # Intialize submodels
        Hmodel = NHNet()
        Hcriterion = nn.MSELoss()
        if torch.cuda.is_available():
            Hmodel = Hmodel.cuda()
            gamma_exp = gamma_exp.cuda()
            Hcriterion = Hcriterion.cuda()
            
        # Start Training
        epoch = 20
        Hoptimizer = optim.Adam(Hmodel.parameters(), lr=0.001)
        for i in range(epoch):
            Hmodel.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                # Create one-hot label target tensor
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                # Calculate F(x)
                output = initial_model(data)
                # Calculate the output from all the models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        output = output.cuda()
                        model = model.cuda()
                    output = output + gamma_exp[j] * model(data)
#                 print("output is:", output)
                #Convert into Onehot label so that it would be able to calculate the residual
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                #Calculate Residual
#                 print("target_onehot is:", target_onehot)
#                 print("output is:", output)
                residual = mseresidual(target_onehot, output)
                houtput = Hmodel(data)
#                 print("houtput is:", houtput)
                residual = residual
#                 print(residual)
                residual = residual.type(torch.cuda.FloatTensor)
                houtput = houtput.type(torch.cuda.FloatTensor)
                #Calculate the loss
                loss = Hcriterion(houtput, residual)
                Hoptimizer.zero_grad()
                loss.backward(retain_graph = True)
                Hoptimizer.step()
                # Print out current Process
#                 if (batch_idx + 1)% 100 == 0 and i % 10 == 0:
                print('Train Epoch: Model Number: {} {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        m+1,i, (batch_idx + 1) * len(data), len(train_loader.dataset),
                        100. * (batch_idx + 1) / len(train_loader), loss.item()))
        models.append(Hmodel)
        
        # Initialize a random gamma
        gamma = torch.rand(1, requires_grad=True, device="cuda")
        Goptimizer = optim.Adam([gamma], lr=0.01)
        Gcriterion = nn.MSELoss()
        # Start finding the best gamma
        for i in range(20):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                    Hmodel = Hmodel.cuda()
                    gamma = gamma.cuda()
                    
                #Calculate the initial output
                Goptimizer.zero_grad()  
                output = initial_model(data)
                #Calculate the final output by combining all previous models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        model = model.cuda()
                        output = output.cuda()
                        gamma_temp = gamma_exp[j]
                        gamma_temp = gamma_temp.cuda()
                    output = output + gamma_temp * model(data)
                # Covert into one-hot label
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                # Get the currect model output
                temp = Hmodel(data)
                # Find the current ensemble model output
                predicted = output + gamma * temp
                # Calculate the loss
                loss = Gcriterion(predicted, target_onehot)
                loss.backward(retain_graph = True)
                # Optimize the gamma
                Goptimizer.step()  
                
        gamma_exp[m] = gamma
    print(gamma_exp)
    return models, gamma_exp

## Start Gradient Boosting Training

In [12]:
num_of_models = 3
models, gamma_exp = GradientBoosting(initial_model, num_of_models)

Train Epoch: Model Number: 1 0 [128/60000 (0%)]	Loss: 0.025713
Train Epoch: Model Number: 1 0 [256/60000 (0%)]	Loss: 0.027314
Train Epoch: Model Number: 1 0 [384/60000 (1%)]	Loss: 0.019532
Train Epoch: Model Number: 1 0 [512/60000 (1%)]	Loss: 0.021344
Train Epoch: Model Number: 1 0 [640/60000 (1%)]	Loss: 0.020048
Train Epoch: Model Number: 1 0 [768/60000 (1%)]	Loss: 0.023707
Train Epoch: Model Number: 1 0 [896/60000 (1%)]	Loss: 0.025426
Train Epoch: Model Number: 1 0 [1024/60000 (2%)]	Loss: 0.023756
Train Epoch: Model Number: 1 0 [1152/60000 (2%)]	Loss: 0.023914
Train Epoch: Model Number: 1 0 [1280/60000 (2%)]	Loss: 0.018082
Train Epoch: Model Number: 1 0 [1408/60000 (2%)]	Loss: 0.016412
Train Epoch: Model Number: 1 0 [1536/60000 (3%)]	Loss: 0.014678
Train Epoch: Model Number: 1 0 [1664/60000 (3%)]	Loss: 0.019313
Train Epoch: Model Number: 1 0 [1792/60000 (3%)]	Loss: 0.020914
Train Epoch: Model Number: 1 0 [1920/60000 (3%)]	Loss: 0.025555
Train Epoch: Model Number: 1 0 [2048/60000 (3%)

Train Epoch: Model Number: 1 0 [16256/60000 (27%)]	Loss: 0.013175
Train Epoch: Model Number: 1 0 [16384/60000 (27%)]	Loss: 0.023572
Train Epoch: Model Number: 1 0 [16512/60000 (28%)]	Loss: 0.017896
Train Epoch: Model Number: 1 0 [16640/60000 (28%)]	Loss: 0.025182
Train Epoch: Model Number: 1 0 [16768/60000 (28%)]	Loss: 0.022293
Train Epoch: Model Number: 1 0 [16896/60000 (28%)]	Loss: 0.023583
Train Epoch: Model Number: 1 0 [17024/60000 (28%)]	Loss: 0.028359
Train Epoch: Model Number: 1 0 [17152/60000 (29%)]	Loss: 0.016297
Train Epoch: Model Number: 1 0 [17280/60000 (29%)]	Loss: 0.020697
Train Epoch: Model Number: 1 0 [17408/60000 (29%)]	Loss: 0.019399
Train Epoch: Model Number: 1 0 [17536/60000 (29%)]	Loss: 0.029945
Train Epoch: Model Number: 1 0 [17664/60000 (29%)]	Loss: 0.019370
Train Epoch: Model Number: 1 0 [17792/60000 (30%)]	Loss: 0.026917
Train Epoch: Model Number: 1 0 [17920/60000 (30%)]	Loss: 0.014833
Train Epoch: Model Number: 1 0 [18048/60000 (30%)]	Loss: 0.020736
Train Epoc

Train Epoch: Model Number: 1 0 [32512/60000 (54%)]	Loss: 0.019318
Train Epoch: Model Number: 1 0 [32640/60000 (54%)]	Loss: 0.019333
Train Epoch: Model Number: 1 0 [32768/60000 (55%)]	Loss: 0.017578
Train Epoch: Model Number: 1 0 [32896/60000 (55%)]	Loss: 0.023764
Train Epoch: Model Number: 1 0 [33024/60000 (55%)]	Loss: 0.022058
Train Epoch: Model Number: 1 0 [33152/60000 (55%)]	Loss: 0.023316
Train Epoch: Model Number: 1 0 [33280/60000 (55%)]	Loss: 0.014808
Train Epoch: Model Number: 1 0 [33408/60000 (56%)]	Loss: 0.023079
Train Epoch: Model Number: 1 0 [33536/60000 (56%)]	Loss: 0.021793
Train Epoch: Model Number: 1 0 [33664/60000 (56%)]	Loss: 0.022253
Train Epoch: Model Number: 1 0 [33792/60000 (56%)]	Loss: 0.029122
Train Epoch: Model Number: 1 0 [33920/60000 (57%)]	Loss: 0.025662
Train Epoch: Model Number: 1 0 [34048/60000 (57%)]	Loss: 0.018114
Train Epoch: Model Number: 1 0 [34176/60000 (57%)]	Loss: 0.022174
Train Epoch: Model Number: 1 0 [34304/60000 (57%)]	Loss: 0.016400
Train Epoc

Train Epoch: Model Number: 1 0 [48768/60000 (81%)]	Loss: 0.017052
Train Epoch: Model Number: 1 0 [48896/60000 (81%)]	Loss: 0.025184
Train Epoch: Model Number: 1 0 [49024/60000 (82%)]	Loss: 0.026619
Train Epoch: Model Number: 1 0 [49152/60000 (82%)]	Loss: 0.019363
Train Epoch: Model Number: 1 0 [49280/60000 (82%)]	Loss: 0.023400
Train Epoch: Model Number: 1 0 [49408/60000 (82%)]	Loss: 0.015626
Train Epoch: Model Number: 1 0 [49536/60000 (83%)]	Loss: 0.017843
Train Epoch: Model Number: 1 0 [49664/60000 (83%)]	Loss: 0.017508
Train Epoch: Model Number: 1 0 [49792/60000 (83%)]	Loss: 0.021360
Train Epoch: Model Number: 1 0 [49920/60000 (83%)]	Loss: 0.019083
Train Epoch: Model Number: 1 0 [50048/60000 (83%)]	Loss: 0.022026
Train Epoch: Model Number: 1 0 [50176/60000 (84%)]	Loss: 0.023727
Train Epoch: Model Number: 1 0 [50304/60000 (84%)]	Loss: 0.019229
Train Epoch: Model Number: 1 0 [50432/60000 (84%)]	Loss: 0.022305
Train Epoch: Model Number: 1 0 [50560/60000 (84%)]	Loss: 0.017753
Train Epoc

Train Epoch: Model Number: 1 1 [6144/60000 (10%)]	Loss: 0.025105
Train Epoch: Model Number: 1 1 [6272/60000 (10%)]	Loss: 0.016027
Train Epoch: Model Number: 1 1 [6400/60000 (11%)]	Loss: 0.027161
Train Epoch: Model Number: 1 1 [6528/60000 (11%)]	Loss: 0.026580
Train Epoch: Model Number: 1 1 [6656/60000 (11%)]	Loss: 0.029877
Train Epoch: Model Number: 1 1 [6784/60000 (11%)]	Loss: 0.020094
Train Epoch: Model Number: 1 1 [6912/60000 (12%)]	Loss: 0.020764
Train Epoch: Model Number: 1 1 [7040/60000 (12%)]	Loss: 0.013819
Train Epoch: Model Number: 1 1 [7168/60000 (12%)]	Loss: 0.021288
Train Epoch: Model Number: 1 1 [7296/60000 (12%)]	Loss: 0.024337
Train Epoch: Model Number: 1 1 [7424/60000 (12%)]	Loss: 0.022121
Train Epoch: Model Number: 1 1 [7552/60000 (13%)]	Loss: 0.017478
Train Epoch: Model Number: 1 1 [7680/60000 (13%)]	Loss: 0.019142
Train Epoch: Model Number: 1 1 [7808/60000 (13%)]	Loss: 0.023377
Train Epoch: Model Number: 1 1 [7936/60000 (13%)]	Loss: 0.017378
Train Epoch: Model Number

...

Train Epoch: Model Number: 3 18 [40832/60000 (68%)]	Loss: 0.021791
Train Epoch: Model Number: 3 18 [40960/60000 (68%)]	Loss: 0.023984
Train Epoch: Model Number: 3 18 [41088/60000 (68%)]	Loss: 0.026156
Train Epoch: Model Number: 3 18 [41216/60000 (69%)]	Loss: 0.020224
Train Epoch: Model Number: 3 18 [41344/60000 (69%)]	Loss: 0.023760
Train Epoch: Model Number: 3 18 [41472/60000 (69%)]	Loss: 0.029972
Train Epoch: Model Number: 3 18 [41600/60000 (69%)]	Loss: 0.017407
Train Epoch: Model Number: 3 18 [41728/60000 (70%)]	Loss: 0.024386
Train Epoch: Model Number: 3 18 [41856/60000 (70%)]	Loss: 0.027568
Train Epoch: Model Number: 3 18 [41984/60000 (70%)]	Loss: 0.026730
Train Epoch: Model Number: 3 18 [42112/60000 (70%)]	Loss: 0.024006
Train Epoch: Model Number: 3 18 [42240/60000 (70%)]	Loss: 0.022070
Train Epoch: Model Number: 3 18 [42368/60000 (71%)]	Loss: 0.027939
Train Epoch: Model Number: 3 18 [42496/60000 (71%)]	Loss: 0.022947
Train Epoch: Model Number: 3 18 [42624/60000 (71%)]	Loss: 0.02

Train Epoch: Model Number: 3 18 [56704/60000 (94%)]	Loss: 0.022979
Train Epoch: Model Number: 3 18 [56832/60000 (95%)]	Loss: 0.022060
Train Epoch: Model Number: 3 18 [56960/60000 (95%)]	Loss: 0.027171
Train Epoch: Model Number: 3 18 [57088/60000 (95%)]	Loss: 0.021734
Train Epoch: Model Number: 3 18 [57216/60000 (95%)]	Loss: 0.022999
Train Epoch: Model Number: 3 18 [57344/60000 (96%)]	Loss: 0.021411
Train Epoch: Model Number: 3 18 [57472/60000 (96%)]	Loss: 0.025443
Train Epoch: Model Number: 3 18 [57600/60000 (96%)]	Loss: 0.020707
Train Epoch: Model Number: 3 18 [57728/60000 (96%)]	Loss: 0.027062
Train Epoch: Model Number: 3 18 [57856/60000 (96%)]	Loss: 0.023667
Train Epoch: Model Number: 3 18 [57984/60000 (97%)]	Loss: 0.016220
Train Epoch: Model Number: 3 18 [58112/60000 (97%)]	Loss: 0.018492
Train Epoch: Model Number: 3 18 [58240/60000 (97%)]	Loss: 0.026748
Train Epoch: Model Number: 3 18 [58368/60000 (97%)]	Loss: 0.023623
Train Epoch: Model Number: 3 18 [58496/60000 (97%)]	Loss: 0.02

Train Epoch: Model Number: 3 19 [13824/60000 (23%)]	Loss: 0.024896
Train Epoch: Model Number: 3 19 [13952/60000 (23%)]	Loss: 0.028792
Train Epoch: Model Number: 3 19 [14080/60000 (23%)]	Loss: 0.020145
Train Epoch: Model Number: 3 19 [14208/60000 (24%)]	Loss: 0.019625
Train Epoch: Model Number: 3 19 [14336/60000 (24%)]	Loss: 0.020755
Train Epoch: Model Number: 3 19 [14464/60000 (24%)]	Loss: 0.019856
Train Epoch: Model Number: 3 19 [14592/60000 (24%)]	Loss: 0.027248
Train Epoch: Model Number: 3 19 [14720/60000 (25%)]	Loss: 0.024557
Train Epoch: Model Number: 3 19 [14848/60000 (25%)]	Loss: 0.021926
Train Epoch: Model Number: 3 19 [14976/60000 (25%)]	Loss: 0.019534
Train Epoch: Model Number: 3 19 [15104/60000 (25%)]	Loss: 0.026356
Train Epoch: Model Number: 3 19 [15232/60000 (25%)]	Loss: 0.028050
Train Epoch: Model Number: 3 19 [15360/60000 (26%)]	Loss: 0.023258
Train Epoch: Model Number: 3 19 [15488/60000 (26%)]	Loss: 0.021091
Train Epoch: Model Number: 3 19 [15616/60000 (26%)]	Loss: 0.02

Train Epoch: Model Number: 3 19 [30592/60000 (51%)]	Loss: 0.024105
Train Epoch: Model Number: 3 19 [30720/60000 (51%)]	Loss: 0.021066
Train Epoch: Model Number: 3 19 [30848/60000 (51%)]	Loss: 0.026475
Train Epoch: Model Number: 3 19 [30976/60000 (52%)]	Loss: 0.025669
Train Epoch: Model Number: 3 19 [31104/60000 (52%)]	Loss: 0.018782
Train Epoch: Model Number: 3 19 [31232/60000 (52%)]	Loss: 0.022537
Train Epoch: Model Number: 3 19 [31360/60000 (52%)]	Loss: 0.020965
Train Epoch: Model Number: 3 19 [31488/60000 (52%)]	Loss: 0.021880
Train Epoch: Model Number: 3 19 [31616/60000 (53%)]	Loss: 0.027775
Train Epoch: Model Number: 3 19 [31744/60000 (53%)]	Loss: 0.023204
Train Epoch: Model Number: 3 19 [31872/60000 (53%)]	Loss: 0.022041
Train Epoch: Model Number: 3 19 [32000/60000 (53%)]	Loss: 0.023750
Train Epoch: Model Number: 3 19 [32128/60000 (54%)]	Loss: 0.021291
Train Epoch: Model Number: 3 19 [32256/60000 (54%)]	Loss: 0.022841
Train Epoch: Model Number: 3 19 [32384/60000 (54%)]	Loss: 0.02

Train Epoch: Model Number: 3 19 [47488/60000 (79%)]	Loss: 0.026886
Train Epoch: Model Number: 3 19 [47616/60000 (79%)]	Loss: 0.020044
Train Epoch: Model Number: 3 19 [47744/60000 (80%)]	Loss: 0.018021
Train Epoch: Model Number: 3 19 [47872/60000 (80%)]	Loss: 0.020571
Train Epoch: Model Number: 3 19 [48000/60000 (80%)]	Loss: 0.024941
Train Epoch: Model Number: 3 19 [48128/60000 (80%)]	Loss: 0.017360
Train Epoch: Model Number: 3 19 [48256/60000 (80%)]	Loss: 0.026714
Train Epoch: Model Number: 3 19 [48384/60000 (81%)]	Loss: 0.024201
Train Epoch: Model Number: 3 19 [48512/60000 (81%)]	Loss: 0.024454
Train Epoch: Model Number: 3 19 [48640/60000 (81%)]	Loss: 0.024293
Train Epoch: Model Number: 3 19 [48768/60000 (81%)]	Loss: 0.021653
Train Epoch: Model Number: 3 19 [48896/60000 (81%)]	Loss: 0.020170
Train Epoch: Model Number: 3 19 [49024/60000 (82%)]	Loss: 0.023685
Train Epoch: Model Number: 3 19 [49152/60000 (82%)]	Loss: 0.024439
Train Epoch: Model Number: 3 19 [49280/60000 (82%)]	Loss: 0.02

## Save or Load submodels and optimized gamma

In [24]:
# Save all the models trained
for i in range(num_of_models):
    model = models[i]
    torch.save(model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(i) + '.pth')

In [25]:
# Save the optimized gamma
torch.save(gamma_exp, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')

In [ ]:
# Load the models from the local files
num_of_models = 3
models = []
for x in range(num_of_models):
    globals()['model%s' % x] = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(1) + '.pth')
    models.append(globals()['model%s' % x])
print(models)

In [ ]:
# Load the optimized gamma from the local files
gamma_exp = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')
print(gamma_exp)

## Ensemble Model Accuracy

In [19]:
#Find the accuracy of the ensemble model
# initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

loss = 0
correct = 0
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = initial_model(data)
        for i in range(num_of_models):
            model = models[i]
            if torch.cuda.is_available():
                model = model.cuda()
                output = output.cuda()
                gamma_temp = gamma_exp[i]
                gamma_temp = gamma_temp.cuda()
            output = output + gamma_temp * model(data)
        loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
loss /= len(train_loader.dataset)  
print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.



Average loss: 1.5063, Accuracy: 57166/60000 (95.277%)



## CW Attack on Intial Model, Testing Robust Accuracy

In [20]:
# Attack the initial model using CW attack
initial_model.eval()

correct = 0
total = 0

cw_attack = CW(initial_model, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 42.52 %


## Creating Ensemble CW Attack

In [21]:
# Create ensemble CW attack
class Attack_ensemble(object):
    r"""
    Base class for all attacks.
    .. note::
        It automatically set device to the device where given model is.
        It temporarily changes the original model's training mode to `test`
        by `.eval()` only during an attack process.
    """
    def __init__(self, name, model, models, gamma):
        r"""
        Initializes internal attack state.
        Arguments:
            name (str) : name of an attack.
            model (torch.nn.Module): model to attack.
        """

        self.attack = name
        self.model = model
        self.models = models
        self.gamma = gamma
        self.model_name = str(model).split("(")[0]

        self.training = model.training
        self.device = next(model.parameters()).device
        
        self._targeted = 1
        self._attack_mode = 'original'
        self._return_type = 'float'

    def forward(self, *input):
        r"""
        It defines the computation performed at every call.
        Should be overridden by all subclasses.
        """
        raise NotImplementedError
        
    def set_attack_mode(self, mode):
        r"""
        Set the attack mode.
  
        Arguments:
            mode (str) : 'original' (DEFAULT)
                         'targeted' - Use input labels as targeted labels.
                         'least_likely' - Use least likely labels as targeted labels.
        """
        if self._attack_mode is 'only_original':
            raise ValueError("Changing attack mode is not supported in this attack method.")
            
        if mode=="original":
            self._attack_mode = "original"
            self._targeted = 1
            self._transform_label = self._get_label
        elif mode=="targeted":
            self._attack_mode = "targeted"
            self._targeted = -1
            self._transform_label = self._get_label
        elif mode=="least_likely":
            self._attack_mode = "least_likely"
            self._targeted = -1
            self._transform_label = self._get_least_likely_label
        else:
            raise ValueError(mode + " is not a valid mode. [Options : original, targeted, least_likely]")
            
    def set_return_type(self, type):
        r"""
        Set the return type of adversarial images: `int` or `float`.
        Arguments:
            type (str) : 'float' or 'int'. (DEFAULT : 'float')
        """
        if type == 'float':
            self._return_type = 'float'
        elif type == 'int':
            self._return_type = 'int'
        else:
            raise ValueError(type + " is not a valid type. [Options : float, int]")

    def save(self, save_path, data_loader, verbose=True):
        r"""
        Save adversarial images as torch.tensor from given torch.utils.data.DataLoader.
        Arguments:
            save_path (str) : save_path.
            data_loader (torch.utils.data.DataLoader) : data loader.
            verbose (bool) : True for displaying detailed information. (DEFAULT : True)
        """
        self.model.eval()

        image_list = []
        label_list = []

        correct = 0
        total = 0

        total_batch = len(data_loader)

        for step, (images, labels) in enumerate(data_loader):
            adv_images = self.__call__(images, labels)

            image_list.append(adv_images.cpu())
            label_list.append(labels.cpu())

            if self._return_type == 'int':
                adv_images = adv_images.float()/255

            if verbose:
                outputs = self.model(adv_images)
                for i in range(len(self.models)):
                    sub_model = self.models[i]
                    outputs = outputs + self.gamma[i] * sub_model(adv_images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(self.device)).sum()

                acc = 100 * float(correct) / total
                print('- Save Progress : %2.2f %% / Accuracy : %2.2f %%' % ((step+1)/total_batch*100, acc), end='\r')

        x = torch.cat(image_list, 0)
        y = torch.cat(label_list, 0)
        torch.save((x, y), save_path)
        print('\n- Save Complete!')

        self._switch_model()
        
    def _transform_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        """
        return labels
        
    def _get_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return input labels.
        """
        return labels
    
    def _get_least_likely_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return least likely labels.
        """
        outputs = self.model(images)
        for i in range(len(self.models)):
            sub_model = self.models[i]
            outputs = outputs + self.gamma[i] * sub_model(images)
        _, labels = torch.min(outputs.data, 1)
        labels = labels.detach_()
        return labels
    
    def _to_uint(self, images):
        r"""
        Function for changing the return type.
        Return images as int.
        """
        return (images*255).type(torch.uint8)

    def _switch_model(self):
        r"""
        Function for changing the training mode of the model.
        """
        if self.training:
            self.model.train()
            for i in range(len(self.models)):
                self.models[i].train()
        else:
            self.model.eval()
            for i in range(len(self.models)):
                self.models[i].eval()

    def __str__(self):
        info = self.__dict__.copy()
        
        del_keys = ['model', 'attack']
        
        for key in info.keys():
            if key[0] == "_" :
                del_keys.append(key)
                
        for key in del_keys:
            del info[key]
        
        info['attack_mode'] = self._attack_mode
        if info['attack_mode'] == 'only_original' :
            info['attack_mode'] = 'original'
            
        info['return_type'] = self._return_type
        
        return self.attack + "(" + ', '.join('{}={}'.format(key, val) for key, val in info.items()) + ")"

    def __call__(self, *input, **kwargs):
        self.model.eval()
        for i in range(len(self.models)):
            self.models[i].eval()
        images = self.forward(*input, **kwargs)
        self._switch_model()

        if self._return_type == 'int':
            images = self._to_uint(images)

        return images

In [22]:
# Create Ensemble CW Attack
import warnings

class CW_Ensemble(Attack_ensemble):
    r"""
    CW in the paper 'Towards Evaluating the Robustness of Neural Networks'
    [https://arxiv.org/abs/1608.04644]
    Distance Measure : L2
        
    Arguments:
        model (nn.Module): model to attack.
        c (float): c in the paper. parameter for box-constraint. (DEFALUT : 1e-4)    
            :math:`minimize \Vert\frac{1}{2}(tanh(w)+1)-x\Vert^2_2+c\cdot f(\frac{1}{2}(tanh(w)+1))`    
        kappa (float): kappa (also written as 'confidence') in the paper. (DEFALUT : 0)
            :math:`f(x')=max(max\{Z(x')_i:i\neq t\} -Z(x')_t, - \kappa)`
        steps (int): number of steps. (DEFALUT : 1000)
        lr (float): learning rate of the Adam optimizer. (DEFALUT : 0.01)
        
    .. warning:: With default c, you can't easily get adversarial images. Set higher c like 1.
    
    Shape:
        - images: :math:`(N, C, H, W)` where `N = number of batches`, `C = number of channels`,        `H = height` and `W = width`. It must have a range [0, 1].
        - labels: :math:`(N)` where each value :math:`y_i` is :math:`0 \leq y_i \leq` `number of labels`.
        - output: :math:`(N, C, H, W)`.
          
    Examples::
        >>> attack = torchattacks.CW(model, targeted=False, c=1e-4, kappa=0, steps=1000, lr=0.01)
        >>> adv_images = attack(images, labels)
        
    .. note:: NOT IMPLEMENTED methods in the paper due to time consuming.
    
        (1) Binary search for c.
        
        (2) Choosing best L2 adversaries.
    """
    def __init__(self, model, models, gamma, c=1e-4, kappa=0, steps=1000, lr=0.01):
        super(CW_Ensemble, self).__init__("CW", model, models, gamma)
        self.c = c
        self.kappa = kappa
        self.steps = steps
        self.lr = lr

    def forward(self, images, labels):
        r"""
        Overridden.
        """
        images = images.to(self.device)
        labels = labels.to(self.device)
        labels = self._transform_label(images, labels)

        # f-function in the paper
        def f(x):
            outputs = self.model(x)
            for i in range(len(self.models)):
                sub_model = self.models[i]
                outputs = outputs + self.gamma[i] * sub_model(x)
            one_hot_labels = torch.eye(len(outputs[0]))[labels].to(self.device)

            i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
            j = torch.masked_select(outputs, one_hot_labels.bool())

            return torch.clamp(self._targeted*(j-i), min=-self.kappa)

        w = torch.zeros_like(images).to(self.device)
        w.detach_()
        w.requires_grad = True

        optimizer = optim.Adam([w], lr=self.lr)
        prev = 1e10

        for step in range(self.steps):

            a = 1/2*(nn.Tanh()(w) + 1)

            loss1 = nn.MSELoss(reduction='sum')(a, images)
            loss2 = torch.sum(self.c*f(a))

            cost = loss1 + loss2

            optimizer.zero_grad()
            cost.backward(retain_graph=True)
            optimizer.step()

            # Early Stop when loss does not converge.
            if step % (self.steps//10) == 0:
                if cost > prev:
                    warnings.warn("Early stopped because the loss is not converged.")
                    return (1/2*(nn.Tanh()(w) + 1)).detach()
                prev = cost

            # print('- CW Attack Progress : %2.2f %%        ' %((step+1)/self.steps*100), end='\r')

        adv_images = (1/2*(nn.Tanh()(w) + 1)).detach()

        return adv_images

## CW Attack Ensemble Model, Testing Robust Accuracy

In [23]:
#Attack the ensemble model
initial_model.eval()
for i in range(num_of_models):
    models[i].eval()

correct = 0
total = 0

cw_attack = CW_Ensemble(model = initial_model,models = models,gamma = gamma_exp, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        for i in range(num_of_models):
            sub_model = models[i]
            outputs = outputs + gamma_exp[i] * sub_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 43.80 %
